# Build YOLOv7 App Pipeline
A script that build a YOLOv7 app to run inference at the edge.

To the left in the file browser pane, you can find the /cloudstorage folder. Under /cloudstorage/models we store different versions of the machine learning models in version-named folders.

When a new version of the sustayn model is created, put it in `/cloudstorage/models/<version>/<all the files>` and update the `model_name` argument below in the Define the pipeline section.

When that is done, run this notebook.
    
## Install prerequisites

In [45]:
# Install the SDK
!pip3 install kfp --upgrade --user --quiet

## Define components

In [59]:
import kfp
from kfp import compiler, dsl, components
from kfp.components import load_component_from_text
from typing import NamedTuple

def get_metadata_args(
    image_name: str,
    model_name: str,
    domain: str,
    namespace: str) -> NamedTuple(
  'PipelineArgs',
  [
    ('project', str),
    ('namespace', str),
    ('registry', str),
    ('bucket_name', str),
    ('image_name', str)
  ]):
    print("Preparing metadata for build {}".format(model_name))
    import urllib.request
    from collections import namedtuple
    
    def getMetadataValue(url):
        req = urllib.request.Request(url)
        req.add_header("Metadata-Flavor", "Google")
        return urllib.request.urlopen(req).read().decode()
    
    url = "http://metadata.google.internal/computeMetadata/v1/project/project-id"
    project_id = getMetadataValue(url)
    print("Project ID {}".format(project_id))

    output = namedtuple('PipelineArgs', ['project', 'namespace', 'registry', 'bucket_name', 'image_name'])
    return output(
        project_id, 
        namespace, 
        f'us-central1-docker.pkg.dev/{project_id}/{namespace}', 
        f'{namespace}.{domain}',
        f'us-central1-docker.pkg.dev/{project_id}/{namespace}/{image_name}'
    )

get_metadata_args_op = components.create_component_from_func(
    get_metadata_args,
    output_component_file='yolov7_get_metadata_args.yaml')

    
def prepare_model_files(
    model_name: str,
    bucket_name: str) -> NamedTuple(
  'ModelFiles',
  [
    ('weights_file', str),
    ('names_file', str)
  ]):
    '''Prepares Model files'''
    print("Preparing model files for {}".format(model_name))
    
    import os
    from google.cloud import storage
    from collections import namedtuple

    weights_file = 'best_weights.pt'
    names_file = 'object.names'

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    
    def download_blob(source_blob_name, destination_file_name):
        """Downloads a blob from the bucket."""
        mount_folder = '/workspace'
        destination_file_name = os.path.join(mount_folder, destination_file_name)
        print("Downloading object {} to {}.".format(source_blob_name, destination_file_name))
        blob = bucket.blob(source_blob_name)
        blob.download_to_filename(destination_file_name)

    weights_blob = f'models/{model_name}/{weights_file}'
    download_blob(weights_blob, weights_file)
    
    names_blob =f'models/{model_name}/{names_file}'
    download_blob(names_blob, names_file)
    
    output = namedtuple('ModelFiles', ['weights_file', 'names_file'])
    return output(weights_file, names_file)

prepare_model_files_op = components.create_component_from_func(
    prepare_model_files,
    packages_to_install=['google-cloud-storage'],
    output_component_file='yolov7_prepare_model_files.yaml')


def prepare_dockerfile(
    base_image: str,
    model_name: str,
    weights_file: str,
    img_size: int,
    names_file: str) -> NamedTuple(
  'DockerFiles',
  [
    ('dockerfile', str)
  ]):
    '''Prepares Dockerfiles and Manifests'''
    print("Preparing docker file for {}".format(model_name))
    
    import os
    from collections import namedtuple
    import requests
    
    def wget(url, file):
        r = requests.get(url, allow_redirects=True)
        print(f'Downloading {url} to {file}')
        open(file, 'wb').write(r.content)
    
    def write_file(file_path, content):
        mount_folder = "/workspace"
        with open(os.path.join(mount_folder, file_path), 'w') as f:
            print(content)
            f.write(content)
    
    arm64v8 = 'arm64v8.Dockerfile'

    arm64v8_template = f"""
FROM {base_image} as base

ENV MODEL_NAME={model_name}
ENV IMG_SIZE={img_size}
ENV WEIGHTS_FILE={weights_file}
ENV WEIGHTS=/usr/src/app/{weights_file}
ADD {weights_file} /usr/src/app/{weights_file}
ENV CLASS_NAMES=/usr/src/app/{names_file}
ADD {names_file} /usr/src/app/{names_file}
"""
    write_file(arm64v8, arm64v8_template)
    
    output = namedtuple('DockerFiles', ['dockerfile'])
    return output(arm64v8)


prepare_dockerfile_op = components.create_component_from_func(
    prepare_dockerfile,
    packages_to_install=['chevron', 'requests'],
    output_component_file='ppe_prepare_dockerfile.yaml')


## Define the pipeline

In [60]:
# import os
import kfp
from kfp import compiler, dsl, components
from typing import NamedTuple
from kubernetes.client.models import V1EnvVar, V1Volume, V1VolumeMount, V1SecurityContext

pipeline_name = 'Build YOLOv7 App'
pipeline_description = 'A pipeline to build custom YOLOv7 Apps'

@dsl.pipeline(
name = pipeline_name,
description = pipeline_description)
def build_yolov7_app(
    image_name,
    model_name,
    img_size,
    namespace,
    domain):
    
    """A pipeline to build custom multi architecture YOLOv7 Apps"""

    mount_folder = f"/workspace"
    
    # A working directory between steps
    wkdirop = dsl.VolumeOp(
        name=f'Create build workspace',
        resource_name=model_name,
        size="500Mi",
        modes=dsl.VOLUME_MODE_RWO
    ).set_caching_options(False)
    
    get_metadata_args = get_metadata_args_op(
        image_name=image_name,
        model_name=model_name,
        namespace=namespace,
        domain=domain
    ).after(wkdirop)

    prepare_model_files = prepare_model_files_op(
        model_name=model_name,
        bucket_name=get_metadata_args.outputs["bucket_name"]
    ).add_pvolumes({mount_folder: wkdirop.volume}).after(get_metadata_args)

    prepare_docker_file = prepare_dockerfile_op(
        base_image='gcr.io/teknoir/yolov7:l4tr32.7.1',        
        model_name=model_name,
        weights_file=prepare_model_files.outputs["weights_file"],
        img_size=img_size,
        names_file=prepare_model_files.outputs["names_file"]
    ).add_pvolumes({mount_folder: wkdirop.volume}).after(prepare_model_files)
    
    build_app = dsl.ContainerOp(
        name="Build and push app image",
        image="gcr.io/kaniko-project/executor:v1.6.0",
        command=["/kaniko/executor"],
        arguments=[
            "--context", "dir:///workspace/",
            "--dockerfile", prepare_docker_file.outputs['dockerfile'], 
            "--destination", f'{get_metadata_args.outputs["image_name"]}:latest',
            "--label", "app_name=yolov7",
            "--label", "model_type=object_detection"
        ],
        pvolumes={mount_folder: wkdirop.volume}
    ).after(prepare_docker_file)

## Compile pipeline

In [61]:
workflow = kfp.compiler.Compiler().compile(pipeline_func=build_yolov7_app, 
                                           package_path='yolov7_build_ds_app.yaml')

## Upload pipeline or new version of pipeline

In [62]:
import uuid
import json

pipeline_version_file = pipeline_file = 'yolov7_build_ds_app.yaml'

client = kfp.Client(namespace='teknoir')
filter = json.dumps({'predicates': [{'key': 'name', 'op': 1, 'string_value': pipeline_name}]})
pipelines = client.pipelines.list_pipelines(filter=filter)

if not pipelines.pipelines:
    pipeline = client.pipeline_uploads.upload_pipeline(pipeline_file, name=pipeline_name, description=pipeline_description)
else:
    pipeline_version_name = pipeline_name + f' - {str(uuid.uuid4())[:6]}'
    pipeline_version = client.pipeline_uploads.upload_pipeline_version(pipeline_version_file,
                                                                       name=pipeline_version_name,
                                                                       pipelineid=pipelines.pipelines[0].id)

ERROR:root:Failed to read a token from file '/var/run/secrets/kubeflow/pipelines/token' ([Errno 2] No such file or directory: '/var/run/secrets/kubeflow/pipelines/token').


## To run a pipeline
Now you can find the new pipeline, or version, in the menu to the left, under Workflow -> Pipelines.

### Run the pipeline from the Devstudio

There is a node, under `pipelines`, that allows you to run the pipeline.
In the node properties, the new pipeline is now listed in the pipeline dropdown.
The arguments are listed when you select pipeline, and there is an option to input default values.

### Pipeline arguments
Arguments example:

```
arguments = {
    'image_name': 'yolov7-ppe-hhsv',
    'model_name': 'ppe-bbox-multiclass-20220907231701674',
    'img_size': 416,
    'namespace': os.getenv('NAMESPACE', 'teknoir-ai'),
    'domain': os.getenv('DOMAIN', 'teknoir.cloud')
}
```

In [51]:
# domain = os.getenv('DOMAIN', 'teknoir.cloud')
# namespace = os.getenv('NAMESPACE', 'teknoir-ai')

# arguments = {
#     'image_name': 'yolov7-ppe-hhsv',
#     'model_name': 'ppe-bbox-multiclass-20220907231701674',
#     'img_size': 416,
#     'namespace': namespace,
#     'domain': domain
# }

# client = kfp.Client(namespace='teknoir')
# run_result = client.create_run_from_pipeline_func(build_yolov7_app, arguments=arguments)